In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install tsfel

We trimmed our data to have 500 samples and since Decision Tree model on raw data gave us not so good results, we will be avoiding this.The features extracted by author on UCI HAR dataset consits of many other signals like gyroscope hence we will be using Decision Tree Model on TSFEL extracted features. Now to maintain consistancy we also extract TSFEL features from the raw data we collected which lets both the datasets with same features.

In [ ]:
#Q1: Train on TSFEL featurised HAR data and test on TSFEL featurised collected data
import pandas as pd
import numpy as np
import os
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, classification_report
import tsfel

# 1. Load Training Data
activity_dirs_train = {
    'LAYING': "/content/drive/MyDrive/combined1/Combined/Train/LAYING",
    'SITTING': "/content/drive/MyDrive/combined1/Combined/Train/SITTING",
    'STANDING': "/content/drive/MyDrive/combined1/Combined/Train/STANDING",
    'WALKING': "/content/drive/MyDrive/combined1/Combined/Train/WALKING",
    'WALKING_DOWNSTAIRS': "/content/drive/MyDrive/combined1/Combined/Train/WALKING_DOWNSTAIRS",
    'WALKING_UPSTAIRS': "/content/drive/MyDrive/combined1/Combined/Train/WALKING_UPSTAIRS"
}

cfg = tsfel.get_features_by_domain()

X_features = []
y_labels = []

for activity, activity_dir in activity_dirs_train.items():
    for file_name in os.listdir(activity_dir):
        file_path = os.path.join(activity_dir, file_name)
        data = pd.read_csv(file_path)

        feats = tsfel.time_series_features_extractor(cfg, data, verbose=0)
        X_features.append(feats.values.flatten())
        y_labels.append(activity)

X_train = np.array(X_features)
y_train = np.array(y_labels)

# 2. Load Your Collected Data

activity_dirs_collected = {
    'LAYING': "/content/drive/MyDrive/combined1/Mycombine/Laying Down",
    'SITTING': "/content/drive/MyDrive/combined1/Mycombine/Sitting",
    'STANDING': "/content/drive/MyDrive/combined1/Mycombine/Standing",
    'WALKING': "/content/drive/MyDrive/combined1/Mycombine/Walking",
    'WALKING_DOWNSTAIRS': "/content/drive/MyDrive/combined1/Mycombine/Descending Stairs",
    'WALKING_UPSTAIRS': "/content/drive/MyDrive/combined1/Mycombine/Climbing Stairs"
}

myX_features = []
myy = []

for activity, activity_dir in activity_dirs_collected.items():
    for file_name in os.listdir(activity_dir):
        file_path = os.path.join(activity_dir, file_name)
        data = pd.read_csv(file_path)
        feats = tsfel.time_series_features_extractor(cfg, data, verbose=0)
        myX_features.append(feats.values.flatten())
        myy.append(activity)

myX = np.array(myX_features)
myy = np.array(myy)

#3. Predict on Your Collected Data

clf = DecisionTreeClassifier(random_state=30)
clf.fit(X_train, y_train)
my_preds = clf.predict(myX)

print("\nResults on Collected Data:")
print("Accuracy :", accuracy_score(myy, my_preds))
print("Precision:", precision_score(myy, my_preds, average='weighted'))
print("Recall   :", recall_score(myy, my_preds, average='weighted'))
print("\nConfusion Matrix:\n", confusion_matrix(myy, my_preds))

/tmp/ipython-input-2706331665.py:30: UserWarning: Using default sampling frequency set in configuration file.
  feats = tsfel.time_series_features_extractor(cfg, data, verbose=0)
/tmp/ipython-input-2706331665.py:30: UserWarning: Using default sampling frequency set in configuration file.
  feats = tsfel.time_series_features_extractor(cfg, data, verbose=0)
/tmp/ipython-input-2706331665.py:30: UserWarning: Using default sampling frequency set in configuration file.
  feats = tsfel.time_series_features_extractor(cfg, data, verbose=0)
/tmp/ipython-input-2706331665.py:30: UserWarning: Using default sampling frequency set in configuration file.
  feats = tsfel.time_series_features_extractor(cfg, data, verbose=0)
/tmp/ipython-input-2706331665.py:30: UserWarning: Using default sampling frequency set in configuration file.
  feats = tsfel.time_series_features_extractor(cfg, data, verbose=0)
/tmp/ipython-input-2706331665.py:30: UserWarning: Using default sampling frequency set in configuration f


Results on Collected Data:
Accuracy : 0.16666666666666666
Precision: 0.027777777777777776
Recall   : 0.16666666666666666

Confusion Matrix:
 [[0 0 0 0 3 0]
 [0 0 0 0 3 0]
 [0 0 0 0 3 0]
 [0 0 0 0 3 0]
 [0 0 0 0 3 0]
 [0 0 0 0 3 0]]


/tmp/ipython-input-2706331665.py:56: UserWarning: Using default sampling frequency set in configuration file.
  feats = tsfel.time_series_features_extractor(cfg, data, verbose=0)
/tmp/ipython-input-2706331665.py:56: UserWarning: Using default sampling frequency set in configuration file.
  feats = tsfel.time_series_features_extractor(cfg, data, verbose=0)
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



The UCI HAR dataset was recorder with multiple device and sensors while we used one mobile. The model expects similar features in both data sets but TSFEL features extracted from UCI HAR dataset could be different and hence the model is not able to learn properly.

Q2: Train and test on raw collected data

In [ ]:

import os
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
activity_dirs = {
    'LAYING': "/content/drive/MyDrive/combined1/Mycombine/Laying Down",
    'SITTING': "/content/drive/MyDrive/combined1/Mycombine/Sitting",
    'STANDING': "/content/drive/MyDrive/combined1/Mycombine/Standing",
    'WALKING': "/content/drive/MyDrive/combined1/Mycombine/Walking",
    'WALKING_DOWNSTAIRS': "/content/drive/MyDrive/combined1/Mycombine/Descending Stairs",
    'WALKING_UPSTAIRS': "/content/drive/MyDrive/combined1/Mycombine/Climbing Stairs"
}
X = []
y = []


for activity, activity_dir in activity_dirs.items():
    file_names = os.listdir(activity_dir)
    for file_name in file_names:
        file_path = os.path.join(activity_dir, file_name)
        data = pd.read_csv(file_path)
        total_acceleration = data.to_numpy().flatten()
        X.append(total_acceleration)
        y.append(activity)

X = np.array(X)
y = np.array(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

clf = DecisionTreeClassifier(random_state=0)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

accuracy1 = accuracy_score(y_test, y_pred)
precision1, recall1, _, _ = precision_recall_fscore_support(y_test, y_pred, average='weighted')
conf_matrix1 = confusion_matrix(y_test, y_pred)

print(f"Accuracy: {accuracy1:.4f}")
print(f"Precision: {precision1:.4f}")
print(f"Recall: {recall1:.4f}")
print("Confusion Matrix:")
print(conf_matrix1)

Accuracy: 0.5000
Precision: 0.3056
Recall: 0.5000
Confusion Matrix:
[[2 0 0 0 0]
 [1 0 1 0 0]
 [0 0 1 0 0]
 [0 0 0 0 0]
 [0 0 0 1 0]]


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Q2:Train and test on TSFEL featurized collected data

In [ ]:
#Q2:Train and test on TSFEL featurized collected data
activity_dirs_collected = {
    'LAYING': "/content/drive/MyDrive/combined1/Mycombine/Laying Down",
    'SITTING': "/content/drive/MyDrive/combined1/Mycombine/Sitting",
    'STANDING': "/content/drive/MyDrive/combined1/Mycombine/Standing",
    'WALKING': "/content/drive/MyDrive/combined1/Mycombine/Walking",
    'WALKING_DOWNSTAIRS': "/content/drive/MyDrive/combined1/Mycombine/Descending Stairs",
    'WALKING_UPSTAIRS': "/content/drive/MyDrive/combined1/Mycombine/Climbing Stairs"
}

myX_features = []
myy = []

for activity, activity_dir in activity_dirs_collected.items():
    for file_name in os.listdir(activity_dir):
        file_path = os.path.join(activity_dir, file_name)
        data = pd.read_csv(file_path)
        cfg = tsfel.get_features_by_domain()
        feats = tsfel.time_series_features_extractor(cfg, data, verbose=0)
        myX_features.append(feats.values.flatten())
        myy.append(activity)

myX = np.array(myX_features)
myy = np.array(myy)

X_train, X_test, y_train, y_test = train_test_split(myX, myy, test_size=0.3, random_state=42)


clf = DecisionTreeClassifier(random_state=0)

clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

accuracy2 = accuracy_score(y_test, y_pred)
precision2 = precision_score(y_test, y_pred, average='weighted')
recall2 = recall_score(y_test, y_pred, average='weighted')
conf_matrix2 = confusion_matrix(y_test, y_pred)

print("accuracy=",accuracy2)
print("precision=",precision2)
print("recall=",recall2)
print("Confusion Matrix:")
print(conf_matrix2)

/tmp/ipython-input-142204246.py:19: UserWarning: Using default sampling frequency set in configuration file.
  feats = tsfel.time_series_features_extractor(cfg, data, verbose=0)
/tmp/ipython-input-142204246.py:19: UserWarning: Using default sampling frequency set in configuration file.
  feats = tsfel.time_series_features_extractor(cfg, data, verbose=0)
/tmp/ipython-input-142204246.py:19: UserWarning: Using default sampling frequency set in configuration file.
  feats = tsfel.time_series_features_extractor(cfg, data, verbose=0)
/tmp/ipython-input-142204246.py:19: UserWarning: Using default sampling frequency set in configuration file.
  feats = tsfel.time_series_features_extractor(cfg, data, verbose=0)
/tmp/ipython-input-142204246.py:19: UserWarning: Using default sampling frequency set in configuration file.
  feats = tsfel.time_series_features_extractor(cfg, data, verbose=0)
/tmp/ipython-input-142204246.py:19: UserWarning: Using default sampling frequency set in configuration file.
 

accuracy= 0.6666666666666666
precision= 0.7222222222222222
recall= 0.6666666666666666
Confusion Matrix:
[[2 0 0 0 0]
 [1 1 0 0 0]
 [0 0 1 0 0]
 [0 0 0 0 1]
 [0 0 0 0 0]]


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Training and then Testing on the raw collected data gave us better results. Here while we extracted features from the collected data the same features are also present in the test data and hence the model works comparitvely better.
We also observe as expected that the model learnt on TSFLE extracted data works better in comparison to the model learnt just on raw collected data.
  